<a href="https://colab.research.google.com/github/DeepthiMakineni/FineTuning_flan-t5/blob/main/google_Key_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving skincare.txt to skincare.txt


In [ ]:
# Read the content of skincare.txt
with open('skincare.txt', 'r', encoding='utf-8') as file:
    content = file.read()

print(content[:500])  # preview first 500 characters

Vitamin C helps fade dark spots and improve skin texture.
Niacinamide regulates sebum and supports the skin barrier.
Retinol reduces fine lines and stimulates collagen production.
Sunscreen protects against UV rays and prevents premature aging.
Hyaluronic acid hydrates and plumps the skin, reducing dryness.
Salicylic acid unclogs pores and prevents acne breakouts.
Ceramides help restore the skin’s natural protective barrier.
Green tea extract calms inflammation and reduces redness.
Peptides prom


In [ ]:
!pip install transformers torch accelerate sentencepiece --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.6 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained(
    "google/flan-t5-base",
    torch_dtype=torch.bfloat16 if device == "cuda" else torch.float32,
    device_map='auto'
)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def extract_keywords(text, max_keywords=15):
    prompt = (f"Extract exactly {max_keywords} distinct keywords or key phrases from the following text:\n\n{text}\n\nKeywords:")
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
    outputs = model.generate(
        **inputs,
        max_length=200,
        do_sample=True,
        temperature=0.9,
        top_k=60,
        top_p=0.95,
        repetition_penalty=1.2,
        num_return_sequences=1
    )
    keywords = tokenizer.decode(outputs[0], skip_special_tokens=True)
    keywords_list = [kw.strip() for kw in keywords.split(",") if kw.strip()]
    return keywords_list

# Test
keywords = extract_keywords(content, max_keywords=15)
print("Extracted Keywords:", keywords)

Extracted Keywords: ['azelaic acid', 'acne', 'moisturizer', 'anti-acne', 'retinol', 'sunscreen']


In [ ]:
!pip install transformers datasets torch accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 854.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset

In [ ]:
from google.colab import files

uploaded = files.upload()  # Upload skincare_keywords_dataset.jsonl manually

Saving skincare.jsonl to skincare.jsonl


In [ ]:
from datasets import load_dataset

# load the dataset from your uploaded file
dataset = load_dataset('json', data_files='skincare.jsonl', split='train')

# split dataset into train and validation sets
dataset = dataset.train_test_split(test_size=0.1)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map='auto', torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32)

In [ ]:
max_input_length = 256
max_output_length = 64

def preprocess(examples):
    inputs = ["Extract keywords: " + text for text in examples['text']]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    labels = tokenizer(examples['keywords'], max_length=max_output_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

tokenized_dataset = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/135 [00:00<?, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./flan-t5-keywords",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    logging_steps=10,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-11-217874a4d43e>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: makinenisaideepthi (makinenisaideepthi-florida-atlantic-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.879200,0.074861
2,0.214900,0.033764
3,0.120800,0.013652
4,0.107000,0.008980
5,0.077400,0.007769


TrainOutput(global_step=85, training_loss=0.2401088041417739, metrics={'train_runtime': 1076.6198, 'train_samples_per_second': 0.627, 'train_steps_per_second': 0.079, 'total_flos': 49971237728256.0, 'train_loss': 0.2401088041417739, 'epoch': 5.0})

In [ ]:
trainer.save_model("./flan-t5-keywords-finetuned")

In [ ]:
from transformers import pipeline

model = AutoModelForSeq2SeqLM.from_pretrained("./flan-t5-keywords-finetuned")
tokenizer = AutoTokenizer.from_pretrained("./flan-t5-keywords-finetuned")
keyword_extractor = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Example inference
from google.colab import files

uploaded = files.upload()

# Extract filename from uploaded file
filename = list(uploaded.keys())[0]

# Read the file content
with open(filename, "r") as file:
    text = file.read()

# Run inference with the loaded text
result = keyword_extractor(f"Extract keywords: {text}", max_length=64)
print("Extracted Keywords:", result[0]['generated_text'])


Device set to use cpu


Saving skincare.txt to skincare (2).txt
Extracted Keywords: vitamin C, niacinamide, retinol, sunscreen, hyaluronic acid, ceramides, green tea extract, peptides, exfoliating, AHAs, moisturizers, double cleansing, bakuchiol
